In [25]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.decomposition import PCA

## Read data

In [2]:
data = pd.read_csv("test_sample.csv")

In [4]:
data.shape

(500, 492)

In [5]:
data.head()

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X482,X483,X484,X485,X486,X487,X488,X489,X490,X491
0,79.876423,-1.245862,-2.640738,-4.445075,-2.596791,0.192209,0.439903,1.009914,3.101967,-2.894164,...,-0.249641,4.421276,-0.096290,3.838207,-0.255293,-0.457279,0.126403,-2.279429,2.079863,-1.048308
1,147.601608,2.155426,-1.653922,1.210217,0.811996,1.146012,0.920832,-0.440953,-2.873925,-2.335624,...,0.008773,-3.792059,-0.466665,0.830561,-4.391146,1.774208,-2.788983,0.591398,-1.577763,-1.138426
2,-66.974081,-0.003307,2.523621,-1.258825,1.929429,1.033411,-1.303365,-1.393066,-0.974599,3.221174,...,-1.745593,6.271241,-3.306594,-1.621729,0.057774,1.862335,4.440217,-0.967720,2.119744,-2.949879
3,-11.337617,-0.651550,3.670091,0.079076,4.006305,0.557802,-0.364852,-1.462042,0.251087,-0.535460,...,-3.110233,2.014395,0.672643,-0.236287,0.161460,-0.227571,-2.696188,0.575367,-0.733765,-0.024031
4,-33.124200,2.056242,-1.443588,-2.287429,2.743887,2.908003,-2.742532,-2.044555,0.037670,-0.834969,...,-1.422660,0.099991,2.572091,-0.940041,1.346845,0.733637,-1.150200,1.036893,3.575096,4.354865


## Step 1

In [7]:
Y = data['Y']  
X = data.drop(['Y'], axis=1)  

In [8]:
n_orig = None  

for j in range(2, 492):  # Starting with 2 regressors up to 491
   
    X_j = X.iloc[:, :j]
    
    # Add a constant to the model for the intercept
    X_j = sm.add_constant(X_j)
    
    # Fit the linear regression model
    model = sm.OLS(Y, X_j).fit()
    
    # Check if the determination coefficient (R-squared) is greater than 0.9
    if model.rsquared > 0.9:
        n_orig = j
        break  

In [9]:
print(f"Smallest number of regressors with R^2 > 0.9: {n_orig}")

Smallest number of regressors with R^2 > 0.9: 343


## Step 2

In [ ]:
def rel_imp_me(X, y): 
    names = X.columns
    ser = pd.Series(index=names)
    lm0 = sm.OLS(y, sm.add_constant(X)).fit()
    for c in names:
        lm = sm.OLS(y, sm.add_constant(X[names.drop(c)])).fit()
        ser[c] = lm0.rsquared - lm.rsquared 
    res = pd.DataFrame(columns=['last', 'first', 'betasq', 'pratt'], index=names)
    res['last'] = ser
    corr = X.apply(lambda x: np.corrcoef(y, x)[0, 1], axis=0)
    res['first'] = corr**2
    sx = X.apply(np.std, axis=0)
    res['betasq'] = (lm0.params[names] * sx / np.std(y))**2
    res['pratt'] = (lm0.params[names] * sx / np.std(y)) * corr
    return res

In [22]:
# Perform PCA on the predictors
pca = PCA().fit(X)
X_pca = pca.transform(X)

# Convert the PCA components to a DataFrame for feature importance calculation
X_pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(X_pca.shape[1])])

# Calculate the relative importance metrics for PCA components
metrics_pca = rel_imp_me(X_pca_df, Y)

# Reorder PCA components based on the 'first' importance measure
ordered_pca_components = metrics_pca.sort_values(by='first', ascending=False).index

# Initialize variables
n_PCA = None
determination_coefficient = None

In [23]:
# Fit regression models with an increasing number of ordered PCA components
for i in range(1, len(ordered_pca_components) + 1):
    # Select the ordered PCA components
    X_ordered_pca_i = X_pca_df[ordered_pca_components[:i]]
    
    # Add a constant to the model for the intercept
    X_ordered_pca_i_with_const = sm.add_constant(X_ordered_pca_i)
    
    # Fit the linear regression model using OLS
    model = sm.OLS(Y, X_ordered_pca_i_with_const).fit()
    
    # Check if the R-squared value is greater than 0.9
    if model.rsquared > 0.9:
        n_PCA = i
        determination_coefficient = model.rsquared
        break  # Stop if we meet the R-squared threshold

if n_PCA is not None:
    model_dimensionality_reduction = n_orig - n_PCA
    print(f"Model Dimensionality Reduction: {model_dimensionality_reduction}")
    print(f"Determination Coefficient (with {n_PCA} ordered PCA components): {determination_coefficient:.5f}")
else:
    print("No sufficient PCA components found to achieve R-squared > 0.9")

Model Dimensionality Reduction: 197
Determination Coefficient (with 146 ordered PCA components): 0.90050


In [24]:
determination_coefficient

0.9005026035563104